The purpose of this notebook is to create example json files (similar to however a backend API would be serving data) to create a prototype dashboard of the next version of the IMPACTS dashboard.

In [7]:
import pathlib
import glob
import os

import argparse
import geojson
import pandas as pd
import json
import random

from datetime import datetime, timedelta

In [8]:
# Helper array for day indexing
aggregateMonths = [0, 31, 60, 91, 121, 152, 182, 213, 244, 274, 305, 335]
nsims = 10000

## Generate data for the quantile table

#json_form...

[{
    Location: 'National',
    Type: 'National',
    Current: {
        tornadoes: {
            10: 5,
            50: 9,
            90: 14
        },
        population: {
            10: 100,
            50: 435,
            90: 1340
        },
        ...
    },
    Climo: {
        tornadoes: {
            10: 3,
            50: 5,
            90: 10
        },
        population: {
            10: 50,
            50: 325,
            90: 900
        },
        ...
    }
},
{
    Location: state1,
    Type: 'State',
    Current: {},
    Climo: {}
}
,
{
    Location: state2,
    Type: 'State',
    Current: {},
    Climo: {}
},
{
    Location: cwa1,
    Type: 'CWA',
    Current: {},
    Climo: {}
}
,
{
    Location: cwa2,
    Type: 'CWA',
    Current: {},
    Climo: {}
},
{
    Location: fema1,
    Type: 'FEMA',
    Current: {},
    Climo: {}
}
,
{
    Location: fema2,
    Type: 'FEMA',
    Current: {},
    Climo: {}
}]

In [9]:
jsonResponse = []

In [10]:
# Function to count tornadoes in each simulation
def torCounter(df,filled_df):
    # Use arbitrary column to get count of entries for each sim (i.e., count of tornadoes)
    torsBySim = df.groupby("sim").count().loc[:,'wfos']
    
    # Merge this count with the original 
    merged = filled_df.merge(torsBySim,how='left',on='sim')
    merged.fillna(value=0,inplace=True)
    merged.rename(columns={'wfos':'tornadoes'},inplace=True)
    return merged

In [19]:
# Read in example psv
file = '../data/output/examples/20110427/psvs/20110427162927_schools_fema_time.psv.gz'
df = pd.read_csv(file, sep="|")

In [20]:
df.head()

,sim,population,distance,rating,states,counties,wfos,hospitals,hospitalbeds,mobileparks,mobilehomes,psubstations,plines,slon,slat,elon,elat,schools,fema,time
0,1,2544,4.4415,3,MS,28081,MEG,0,0,0,0,0,0,-88.6066,34.3619,-88.5379,34.3921,0,4,13
1,1,624,3.9692,2,KY,"21051,21121",JKL,0,0,0,0,0,0,-83.8111,37.0729,-83.7601,37.1134,2,4,20
2,1,3280,27.0073,4,"AL,GA","1019,13055,13115","BMX,FFC",0,0,0,0,3,3,-85.5258,34.2540,-85.1459,34.4873,1,"4,4",16
3,1,928,1.1927,0,AL,1089,HUN,0,0,0,0,0,0,-86.4543,35.0071,-86.4403,35.0201,0,4,9
4,1,6464,0.1907,0,PA,42049,CLE,0,0,0,0,0,0,-79.9165,42.1845,-79.9137,42.1863,0,3,11


In [21]:
sims = df.groupby("sim")
fields = sims.sum().loc[:,('population','hospitals','schools','mobilehomes','psubstations')]

# Fill missing sims with 0s (artifact of how pas writes out files)
fill_fields = fields.reindex(list(range(1,nsims+1)),fill_value=0)

In [22]:
# Get tornado counts
merged = torCounter(df,fill_fields)

In [23]:
quants = merged.quantile(q=[0.1,0.5,0.9],interpolation='nearest')
quants

,population,hospitals,schools,mobilehomes,psubstations,tornadoes
0.1,83728,1,30,425,24,45
0.5,190000,4,70,1150,53,82
0.9,391664,11,145,2500,105,154


In [24]:
nationalResponse = {
    'Location': 'National',
    'Type': 'National',
    'Current': {},
    'Climo': {}
}

for name in quants.columns:
    
    nationalResponse['Current'][name] = {}
    nationalResponse['Climo'][name] = {}
    
    for quant in quants.index:
        
        nationalResponse['Current'][name][quant] = int(quants.loc[quant,name])
        # Just generate random numbers for climo until we have the necessary numbers
        nationalResponse['Climo'][name][quant] = random.random()*10

In [25]:
jsonResponse.append(nationalResponse)

### FEMA

In [27]:
femaBrokenOut = df.assign(category=df['fema'].str.split(',')).explode('category').reset_index(drop=True)

# Remove row if state is NaN
femaBrokenOut = femaBrokenOut[femaBrokenOut['category'].notna()]

# Grab a list of the unique states in the simulation
femaImpacted = femaBrokenOut['category'].unique().tolist()

In [28]:
for fema in femaImpacted:
    
    femaResponse = {
        'Location': fema,
        'Type': 'FEMA',
        'Current': {},
        'Climo': {}
    }
    
    ind_fema = femaBrokenOut[femaBrokenOut['category'] == fema]
    #print(f'State: {state}')
    ind_fema_group = ind_fema.groupby("sim").sum().loc[:,['population','hospitals','schools','mobilehomes','psubstations']]
    ind_fema_tot = ind_fema_group.reindex(list(range(1,10001)),fill_value=0)
    
    # Get tor counts for each state
    ind_fema_tot = torCounter(ind_fema,ind_fema_tot)
    
    fema_quants = ind_fema_tot.quantile(q=[0.1,0.5,0.9],interpolation='nearest')
    
    for name in fema_quants.columns:
    
        femaResponse['Current'][name] = {}
        femaResponse['Climo'][name] = {}

        for quant in fema_quants.index:

            femaResponse['Current'][name][quant] = int(fema_quants.loc[quant,name])
            # Just generate random numbers for climo until we have the necessary numbers
            femaResponse['Climo'][name][quant] = random.random()*10
            
    jsonResponse.append(femaResponse)

### States

In [37]:
stBrokenOut = df.assign(category=df['states'].str.split(',')).explode('category').reset_index(drop=True)

# Remove row if state is NaN
stBrokenOut = stBrokenOut[stBrokenOut['category'].notna()]

# Grab a list of the unique states in the simulation
statesImpacted = stBrokenOut['category'].unique().tolist()

In [38]:
for state in statesImpacted:
    
    stateResponse = {
        'Location': state,
        'Type': 'State',
        'Current': {},
        'Climo': {}
    }
    
    ind_state = stBrokenOut[stBrokenOut['category'] == state]
    #print(f'State: {state}')
    ind_state_group = ind_state.groupby("sim").sum().loc[:,['population','hospitals','schools','mobilehomes','psubstations']]
    ind_state_tot = ind_state_group.reindex(list(range(1,10001)),fill_value=0)
    
    # Get tor counts for each state
    ind_state_tot = torCounter(ind_state,ind_state_tot)
    
    st_quants = ind_state_tot.quantile(q=[0.1,0.5,0.9],interpolation='nearest')
    
    for name in st_quants.columns:
    
        stateResponse['Current'][name] = {}
        stateResponse['Climo'][name] = {}

        for quant in st_quants.index:

            stateResponse['Current'][name][quant] = int(st_quants.loc[quant,name])
            # Just generate random numbers for climo until we have the necessary numbers
            stateResponse['Climo'][name][quant] = random.random()*10
            
    jsonResponse.append(stateResponse)

### CWAs

In [48]:
cwaBrokenOut = df.assign(category=df['wfos'].str.split(',')).explode('category').reset_index(drop=True)

# Remove row if state is NaN
cwaBrokenOut = cwaBrokenOut[cwaBrokenOut['category'].notna()]

# Grab a list of the unique states in the simulation
cwasImpacted = cwaBrokenOut['category'].unique().tolist()

In [49]:
for cwa in cwasImpacted:
    
    cwaResponse = {
        'Location': cwa,
        'Type': 'CWA',
        'Current': {},
        'Climo': {}
    }
    
    ind_cwa = cwaBrokenOut[cwaBrokenOut['category'] == cwa]
    ind_cwa_group = ind_cwa.groupby("sim").sum().loc[:,['population','hospitals','schools','mobilehomes','psubstations']]
    ind_cwa_tot = ind_cwa_group.reindex(list(range(1,10001)),fill_value=0)
    
    # Get tor counts for each state
    ind_cwa_tot = torCounter(ind_cwa,ind_cwa_tot)
    
    cwa_quants = ind_cwa_tot.quantile(q=[0.1,0.5,0.9],interpolation='nearest')
    
    for name in cwa_quants.columns:
    
        cwaResponse['Current'][name] = {}
        cwaResponse['Climo'][name] = {}

        for quant in cwa_quants.index:

            cwaResponse['Current'][name][quant] = int(cwa_quants.loc[quant,name])
            # Just generate random numbers for climo until we have the necessary numbers
            cwaResponse['Climo'][name][quant] = random.random()*10
            
    jsonResponse.append(cwaResponse)

In [56]:
with open('../data/output/examples/20110427/processed/jsonResponse_schoolsfema.json','w') as fp:
    json.dump(jsonResponse, fp)